# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d25/391a2922ad597ba080f4b99dea6d62842562d64845ef5df1a384561e.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Delhi, National Capital Territory of Delhi, India**, and the stations the data comes from are shown on the map below.

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd
import numpy as np



def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(25,'391a2922ad597ba080f4b99dea6d62842562d64845ef5df1a384561e')

ModuleNotFoundError: No module named 'mplleaflet'

In [9]:
weather_data = pd.read_csv('data/C2A2_data/BinnedCsvs_d25/391a2922ad597ba080f4b99dea6d62842562d64845ef5df1a384561e.csv')
data_copy = weather_data.copy()
data_copy['Date'] = pd.to_datetime(data_copy['Date'])
data_copy = data_copy.sort('Date')
data_copy = data_copy[:-18264] # remove 2015 data 
day_list = []
month_list = []

for i in data_copy['Date']:
    day_list.append(i.day)
    month_list.append(i.month)
data_copy['Day'] = day_list
data_copy['Month'] = month_list

data_copy_new = data_copy.ix[~((data_copy['Month'] == 2) & (data_copy['Day'] == 29))] # remove leap year day  
data_copy_new['Data_Value'] = list(map(lambda x: x/10, data_copy_new['Data_Value']))

data_copy_min_max = data_copy_new.groupby(['Month','Day'])['Data_Value'].agg({'min':np.min,'max':np.max})
data_copy_min_max


NameError: name 'pd' is not defined

In [ ]:
plt.figure()
plt.plot(list(data_copy_min_max['min']),'-',label = 'Avg Min Temp 2005-2014')
plt.plot(list(data_copy_min_max['max']),'-',label = 'Avg Max Temp 2005-2014')
plt.title('Avg Low & High Temp in 2005-2014 & Record Breaking Temp in 2015')

In [ ]:
plt.gca().fill_between(range(len(list(data_copy_min_max['min']))),
                (list(data_copy_min_max['min'])),
                (list(data_copy_min_max['max'])),
                facecolor = 'c',
                alpha = 0.25)

In [ ]:
import datetime as dt

ax = plt.gca() 
m = ['Jan','Feb','Mar','Apr','May','June','July','Aug','Sept','Oct','Nov','Dec']

# Following is the vital step. It will create a list of day numbers corresponding to middle of each month i.e. 15(Jan), 46(Feb), ... 
ticks = [(dt.date(2017,m,1)-dt.date(2016,12,15)).days for m in range(1,13)]
# It is important to use a non-leap year for this calculation (I used 2017).
# Also, I used (2016,12,15) to substract so that I get middle of each month rather than beginning, it just looks better that way.

ax.set_xticks(ticks)
ax.set_xticklabels(m)
y_ticks_list  = []
for i in ax.get_yticks():
    y_ticks_list.append(str(i) + '\xb0' + 'C')
ax.yaxis.set_ticklabels(y_ticks_list)

In [ ]:
data_copy_2 = weather_data.copy()
data_copy_2['Date'] = pd.to_datetime(data_copy_2['Date'])
data_copy_2 = data_copy_2.sort('Date')
data_copy_2 = data_copy_2[-18264:-1]
day_list_2 = []
month_list_2 = []

for i in data_copy_2['Date']:
    day_list_2.append(i.day)
    month_list_2.append(i.month)
data_copy_2['Day'] = day_list_2
data_copy_2['Month'] = month_list_2
data_copy_2 = data_copy_2.ix[~((data_copy_2['Month'] == 2) & (data_copy_2['Day'] == 29))] # remove leap year day  
data_copy_2['Data_Value'] = list(map(lambda x: x/10, data_copy_2['Data_Value']))
data_copy_min_max_2 = data_copy_2.groupby(['Month','Day'])['Data_Value'].agg({'min':np.min,'max':np.max})
data_copy_min_max['2015_min'] = data_copy_min_max_2['min'] 
data_copy_min_max['2015_max'] = data_copy_min_max_2['max']
data_copy_min_max = data_copy_min_max.reset_index()
data_copy_min_max

In [ ]:
low_2015_list = list(data_copy_min_max[data_copy_min_max['2015_min'] < data_copy_min_max['min']]['2015_min'])

index_low_2015_list = list(data_copy_min_max[data_copy_min_max['2015_min'] < data_copy_min_max['min']].index.values)

print(str(len(low_2015_list)) + ' ' + str(len(index_low_2015_list)))


In [ ]:
plt.scatter(index_low_2015_list, low_2015_list, c = 'blue', label = 'Colder in 2015')

In [ ]:
high_2015_list = list(data_copy_min_max[data_copy_min_max['2015_max'] > data_copy_min_max['max']]['2015_max'])
index_high_2015_list = list(data_copy_min_max[data_copy_min_max['2015_max'] > data_copy_min_max['max']].index.values)
print(str(len(high_2015_list)) + ' ' + str(len(index_high_2015_list)))


In [ ]:
plt.scatter(index_high_2015_list, high_2015_list, c='red',label = 'Hotter in 2015')

In [ ]:
plt.legend(loc = 8, fontsize = 'x-small')